In [ ]:
import pathlib
import itertools

import pandas as pd
from IPython.display import display
import yaml

import dlsproc.bundle
import dlsproc.hier
import dlsproc.postprocess
import dlsproc.assemble

So that *all* the columns of a `pd.DataFrame` are shown:

In [ ]:
pd.set_option('display.max_columns', None)

# Parameters

The directory containing downloaded data:

In [ ]:
data_directory = pathlib.Path.cwd().parent / 'data'
assert data_directory.exists()
print(data_directory)

/home/manu/dlsproc/data


Directory in which the data is to be saved

In [ ]:
# output_directory = pathlib.Path.cwd()
output_directory = data_directory / 'agregados'
assert output_directory.exists()
print(output_directory)

/home/manu/dlsproc/data/agregados


The name of the file **without** extension

In [ ]:
output_file = output_directory / '2018-2021'
print(f'{output_file} (existing? {output_file.exists()})')

/home/manu/dlsproc/data/agregados/2018-2021 (existing? False)


# Processing

> Licitaciones publicadas en la Plataforma mediante mecanismos de agregación, excluyendo los contratos menores

The directory for *outsiders* data

In [ ]:
outsiders_directory = data_directory / 'agregados'
# outsiders_directory = pathlib.Path.cwd().parent / 'samples' / 'yearly'
assert outsiders_directory.exists()
print(outsiders_directory)

/home/manu/dlsproc/data/agregados


We infer whether we are working on a sample or the full dataset (useful later on)

In [ ]:
working_on_a_sample = outsiders_directory.parts[-2] == 'samples'
working_on_a_sample

False

The list of files to be processed. They were downloaded [here](https://www.hacienda.gob.es/es-ES/GobiernoAbierto/Datos%20Abiertos/Paginas/LicitacionesAgregacion.aspx). One could use something like
```
wget https://contrataciondelsectorpublico.gob.es/sindicacion/sindicacion_1044/PlataformasAgregadasSinMenores_2018.zip
```

In [ ]:
zip_files = [
    'PlataformasAgregadasSinMenores_2018.zip',
    'PlataformasAgregadasSinMenores_2019.zip',
    'PlataformasAgregadasSinMenores_2020.zip',
    'PlataformasAgregadasSinMenores_2021.zip'
]

Accumulators for the data itself (contracts) and records of deleted entries

In [ ]:
res_df = None
res_deleted_series = None

Every file is processed in a loop

In [ ]:
%%time
for filename in zip_files:
    
    print(f'Processing "{filename}"')
    
    # above file names are relative to a directory
    f = outsiders_directory / filename
    
    assert f.exists()
    
    # data is read from the above *zip* file, and concatenated into a single `pd.DataFrame`...
    yearly_df = dlsproc.bundle.read_zip(f, concatenate=True)
    
    # ...and re-structured with multiindexed columns
    yearly_df = dlsproc.hier.flat_df_to_multiindexed_df(yearly_df)
    
    # # the same contract might show up more than once due to updates...but only the last one is kept
    # last_update_only_df = dlsproc.postprocess.keep_updates_only(yearly_df)
    
    # the same zip file also contains information (at the beginning) about deleted entries
    deleted_series = dlsproc.bundle.read_deleted_zip(f)
    
    # if this is NOT the first iteration...
    if res_df is not None:
    
        # ...the new data is stacked
        # res_df = dlsproc.assemble.stack(res_df, last_update_only_df)
        res_df = dlsproc.assemble.stack(res_df, yearly_df)
        res_deleted_series = pd.concat((res_deleted_series, deleted_series), axis=0)
        
    # ...if this is the first iteration
    else:
        
        # ...the new data is set as the accumulated result
        # res_df = last_update_only_df
        res_df = yearly_df
        res_deleted_series = deleted_series

Processing "PlataformasAgregadasSinMenores_2018.zip"
Processing "PlataformasAgregadasSinMenores_2019.zip"
Processing "PlataformasAgregadasSinMenores_2020.zip"
Processing "PlataformasAgregadasSinMenores_2021.zip"
CPU times: user 7min 40s, sys: 1.99 s, total: 7min 42s
Wall time: 7min 42s


In [ ]:
res_df.head(2)

id  \
                                                                                               
                                                                                               
                                                                                               
                                                                                               
                                                                                               
                                                                                               
                                                                                               
file name                           entry                                                      
PlataformasAgregadasSinMenores.atom 0      https://contrataciondelestado.es/sindicacion/P...   
                                    1      https://contrataciondelestado.es/sindicacion/P...   

                                                                                     summary  \
                                                                                               
                                                                                               
                                                                                               
                                                                                               
                                                                                               
                                                                                               
                                                                                               
file name                           entry                                                      
PlataformasAgregadasSinMenores.atom 0      Id licitación: OBR-18-058; Órgano de Contratac...   
                                    1      ID licitación: 1279185566111;Órgano de Contrat...   

                                                                                       title  \
                                                                                               
                                                                                               
                                                                                               
                                                                                               
                                                                                               
                                                                                               
                                                                                               
file name                           entry                                                      
PlataformasAgregadasSinMenores.atom 0      Acondicionamiento de caminos en Rañadoiro (TINEO)   
                                    1      servicio de asistencia en materia de comunicac...   

                                                                   updated  \
                                                                             
                                                                             
                                                                             
                                                                             
                                                                             
                                                                             
                                                                             
file name                           entry                                    
PlataformasAgregadasSinMenores.atom 0     2019-01-02 15:18:17.649000+00:00   
                                    1     2019-01-02 15:11:35.118000+00:00   

                                          ContractFolderStat

How many ros and columns?

In [ ]:
res_df.shape

(724706, 42)

However, some contracts show up more than once, and only the last update is to be kept

In [ ]:
res_last_update_only_df = dlsproc.postprocess.keep_updates_only(res_df)
res_last_update_only_df.shape

(181904, 42)

The *deleted series* might contain duplicates. We build a new `pd.Series` dropping duplicates by keeping only the last one.

In [ ]:
# deduplicated_deleted_series = res_deleted_series.sort_values().groupby(res_deleted_series.index).tail(1)
deduplicated_deleted_series = dlsproc.postprocess.deduplicate_deleted_series(res_deleted_series)
deduplicated_deleted_series

file name                                              id                                                                                 
PlataformasAgregadasSinMenores_20210121_030014_1.atom  https://contrataciondelestado.es/sindicacion/PlataformasAgregadasSinMenores/1954167   2021-01-19 23:11:19.614000+00:00
                                                       https://contrataciondelestado.es/sindicacion/PlataformasAgregadasSinMenores/1954168   2021-01-19 23:11:18.304000+00:00
                                                       https://contrataciondelestado.es/sindicacion/PlataformasAgregadasSinMenores/1954169   2021-01-19 23:11:19.821000+00:00
                                                       https://contrataciondelestado.es/sindicacion/PlataformasAgregadasSinMenores/1954170   2021-01-19 23:11:18.345000+00:00
                                                       https://contrataciondelestado.es/sindicacion/PlataformasAgregadasSinMenores/1954171   2021-01-19 23:11:18.5530

In [ ]:
stateful_df = dlsproc.assemble.merge_deleted(res_last_update_only_df, deduplicated_deleted_series)
stateful_df

id  \
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
file name                                          entry                                                      
PlataformasAgregadasSinMenores_20180217_180137_... 453    https://contrataciondelestado.es/sindicacion/P...   
                                                   452    https://contrataciondelestado.es/sindicacion/P...   
                                                   451    https://contrataciondelestado.es/sindicacion/P...   
                                                   448    https://contrataciondelestado.es/sindicacion/P...   
                                                   447    https://contrataciondelestado.es/sindicacion/P...   
..                                                                                                      ...   
PlataformasAgregadasSinMenores.atom                4      https://contrataciondelestado.es/sindicacion/P...   
                                                   3      https://contrataciondelestado.es/sindicacion/P...   
                                                   2      https://contrataciondelestado.es/sindicacion/P...   
                                                   1      https://contrataciondelestado.es/sindicacion/P...   
                                                   0      https://contrataciondelestado.es/sindicacion/P...   

                                                                                                    summary  \
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
file name                                          entry                                                      
PlataformasAgregadasSinMenores_20180217_180137_... 453    Expediente: 1284/17, Entidad: Diputación Provi...   
                                                   452    Expediente: 1282/17, Entidad: Diputación Provi...   
                                                   451    Expediente: 1281/17, Entidad: Diputación Provi...   
                                                   448    Id licitación: 2017017;  Órgano de Contratació...   
                                                   447    Id licitación: B2017002;  Órgano de Contrataci...   
..                                                                                                      ...   
PlataformasAgregadasSinMenores.atom                4      Id licitación: 8044_30146-0-21; Órgano de Cont...   
                                                   3      Id licitación: 3069_1/2021; Órgano de Contrata... 

The number of filled-in rows for column `deleted_on` should match the number of `id`s in `deduplicated_deleted_series` that show up in `stateful_df`

In [ ]:
assert stateful_df['deleted_on'].notna().sum() == len(set(stateful_df['id']) & set(deduplicated_deleted_series.index.get_level_values(1)))

Saving to *pickle*

In [ ]:
if not working_on_a_sample:
    stateful_df.to_pickle(output_file.with_suffix('.pickle'))

Saving to *parquet*

In [ ]:
parquet_df = dlsproc.assemble.parquet_amenable(stateful_df)
parquet_df.head(2)

id  \
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
file name                                          entry                                                      
PlataformasAgregadasSinMenores_20180217_180137_... 453    https://contrataciondelestado.es/sindicacion/P...   
                                                   452    https://contrataciondelestado.es/sindicacion/P...   

                                                                                                    summary  \
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
file name                                          entry                                                      
PlataformasAgregadasSinMenores_20180217_180137_... 453    Expediente: 1284/17, Entidad: Diputación Provi...   
                                                   452    Expediente: 1282/17, Entidad: Diputación Provi...   

                                                                                                      title  \
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
file name                                          entry                                                      
PlataformasAgregadasSinMenores_20180217_180137_... 453    Refuerzo de Firme en la VP 3001 Renedo de Esgu...   
                                                   452    Refuerzo de Firme en la VP 6603 Mota del Marqu...   

                                                                                  updated  \
                                                                                            
                                                                                            
                                                                                            
                                                                   

In [ ]:
if not working_on_a_sample:
    parquet_df.to_parquet(output_file.with_suffix('.parquet'))

## Making a sample

If we are working with sample files, a *parquet* sample file is created

In [ ]:
if working_on_a_sample:
    sample_df  = parquet_df.iloc[itertools.chain(range(10), range(-10, 0))]
    sample_df.to_parquet(outsiders_directory.parent / '2018-2021_20samples.parquet')

In [ ]:
# stateful_df = pd.read_pickle(output_file.with_suffix('.pickle'))
# stateful_df.shape